<a href="https://colab.research.google.com/github/MicheleBonus/cpclab_notebooks/blob/main/get_funpdbe_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Run this code cell first
import requests
import pandas as pd

def fetch_data_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while fetching data: {e}")

def get_funpdbe_annotations(pdb_id):
    url = f"https://www.ebi.ac.uk/pdbe/graph-api/pdb/funpdbe_annotation/{pdb_id}"
    funpdbe_annotations_json = fetch_data_from_url(url)
    return funpdbe_annotations_json


In [ ]:
pdb_id = "4bkx"
funpdbe_annotations_response = get_funpdbe_annotations(pdb_id)

# Dig through to the actual data we want (data from the 3Dcomplex server)
annotations_3dcomplex = [dataset for dataset in funpdbe_annotations_response[pdb_id] if dataset["origin"] == "3Dcomplex"][0]["annotations"]

# Create a dictionary for your dataframes
residues_data = {
    "interface": annotations_3dcomplex[2]["site_residues"],
    "core": annotations_3dcomplex[3]["site_residues"],
    "rim": annotations_3dcomplex[4]["site_residues"],
    "support": annotations_3dcomplex[5]["site_residues"],
    "interior": annotations_3dcomplex[6]["site_residues"],
    "surface": annotations_3dcomplex[7]["site_residues"]
}

# Add class column and concatenate
all_residues = []

for residue_class, data in residues_data.items():
    df = pd.DataFrame(data)
    df['class'] = residue_class
    all_residues.append(df)

final_dataframe = pd.concat(all_residues, axis=0, ignore_index=True)

In [ ]:
final_dataframe